# Applied Seismology, GEOS 626, University of Alaska Fairbanks

- this program computes the toroidal modes for a spherical shell defined by rigidity and density.
- calls these functions: surf_stress, stress_disp_tor, earthfun

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import warnings

from earthfun import earthfun
from scipy.optimize import brentq
from stress_disp_tor import stress_disp_tor
from surf_stress import surf_stress

In [ ]:
# script settings

warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = 12, 8
plt.rcParams['lines.linewidth'] = 1

In [ ]:
# Define constants:
# Run again after each time you do calculations to reset!!

# shell spans from core-mantle boundary (b) to surface (a)
earthr = 6371000.0               # radius of earth, in meters (a)
cmbr = 3480000.0                 # radius of core-mantle boundary, in meters (b)
rspan = np.array([cmbr, earthr]) # [b a]

# Earth model
imod = 0             # Index for earth model (see earthfun.py)
                     # = 0 for homogeneous
                     # = 1 for linear rho(r) and mu(r)
                     # = 2 for cubic rho(r) and mu(r)
rho = 4380           # density, kg/m3
mu  = 5930*5930*rho  # rigidity (mu = 1.54e11 Pa)

# options for searching solution space
l = 2                # degree (l >= 1)
nmax = 8             # maximum n (default = 8)
                     # nmax+1 is the max number of roots/eigenfunctions/subplots
                     # nmax=0 will return the first root (n=0)                     
iplot_eig_funs = 1   # plot eigenfunctions (=1) or not (=0)
                     # =0 will speed up the calculations

# range of frequencies (note: omega = 2*pi*f), in Hz
fmin = 1/3600        # the initial frequency to start (period = one hour)
df = 0.0002          # the frequency step size (chosen by trial and error)
#fmax = 0.08         # stopping frequency (somewhat arbitrary)
fmax = 0.003
fvec = np.arange(fmin,fmax,df)
numf = len(fvec)

max_step = 5e4       # max step for ODE solver

print('frequency vector ranges from %.3f mHz to %.3f mHz'% (fmin*1e3,fmax*1e3))
print('num frequency points is %i, df = %.3f mHz'% (numf,df*1e3))
print('--> period ranges from %.2f min to %.2f min'% (1/fmin/60,1/fmax/60))

plt.close('all')

In [ ]:
# Plot all eigenfunctions, including those that do not satisfy the boundary conditions.
# This block is for initial plotting only!
if True:
    for ii in range(numf):
        print('%2i/%2i: f = %.3f mHz'% (ii+1,numf,fvec[ii]*1e3))
        
        if ii == 0:
            fig2, ax2 = plt.subplots(num=2)
            fig3, ax3 = plt.subplots(num=3)
        
        # update W(r) and T(r), stored within WT
        _, WT, rvec = surf_stress(fvec[ii], l, imod, rho, mu, rspan, max_step, return_wt_rvec=True)
        
        # plotting parameters (radius converted to km)
        rplot = rvec / 1000
        xmx = 1.1
        ymn = rspan[0]/1000
        ymx = rspan[1]/1000
        dy = 100
        
        # displacement for each frequency
        Wplot = WT[0,:]/max(abs(WT[0,:]))
        ax2.plot(Wplot,rplot,'b')
        ax2.plot([0,0],rspan/1000,'k',lw=2)
        ax2.text(Wplot[-1],rplot[-1]+dy/2,str(ii+1))
        ax2.axis([-xmx, xmx, ymn-dy, ymx+2*dy])
        ax2.set(xlabel='normalized displacement, W(r)',ylabel='radius, km')
        
        # stress for each frequency
        Tplot = WT[1,:]/max(abs(WT[1,:]))
        ax3.plot(Tplot,rplot,'r')
        ax3.plot([0,0],rspan/1000,'k',lw=2)
        ax3.text(Tplot[-1],rplot[-1]+dy/2,str(ii+1))
        ax3.axis([-xmx, xmx, ymn-dy, ymx+2*dy])
        ax3.set(xlabel='normalized stress, T(r)',ylabel='radius, km')
        
plt.show()

In [ ]:
# initial freqeuncy and corresponding surface stress
# note: surf_stress calls stress_disp_tor, which depends on degree l
ii = 0
f = fvec[ii]
Tsurf = surf_stress(f, l, imod, rho, mu, rspan, max_step)
n = 0   # counter for n (n=0 is the first root)

print('freq %3i/%3i %10.3e %10.3e %.2f mHz %.1f s %.2f min'% (ii+1, numf, np.nan, Tsurf, f*1e3, 1/f, 1/f/60))

# leave gap for T(n=0,l=1), which do not exist
# note: these are useful when looping over degree l
if l == 1 and nmax > 0: # fill the n >= 1 entries
    n = 1
            
if l == 1 and nmax == 0: # Breaks 
    raise Exception('mode 0T1 does not exist - check your l and nmax value')

froots = np.full((nmax+1,1),np.nan)

# Initialize figure
if iplot_eig_funs == 1:
    if nmax == 0:
        fig, ax = plt.subplots(nrows=1,ncols=1)
    else:
        fig, ax = plt.subplots(nrows=3,ncols=3)

# THIS IS THE KEY LOOP OVER FREQUENCIES
for ii in range(1,len(fvec)):  # loop over the frequencies
    oldf = f
    f = fvec[ii]
            
    # compute the surface stress for this frequency
    Tsurfold = Tsurf
    Tsurf = surf_stress(f, l, imod, rho, mu, rspan, max_step)

    print('freq %3i/%3i %10.3e %10.3e %.2f mHz %.1f s %.2f min'% (ii+1, numf, Tsurfold, Tsurf, f*1e3, 1/f, 1/f/60))
    
    if Tsurfold * Tsurf < 0:
        f0 = brentq(surf_stress,oldf,f, args=(l, imod, rho, mu, rspan, max_step))
        froots[n,0] = f0    # nth entry of froots vector
        print('T(earthr)=0 --> n=%i %.3f mHz l=%i'% (n,f0*1e3,l))
                
        # update eigenfunctions (WT, rvec) for the exact frequency
        _, WT, rvec = surf_stress(f0, l, imod, rho, mu, rspan, max_step, return_wt_rvec=True)
        
        # plotting eigenfunctions (displacement and stress as a function of radius)
        
        if iplot_eig_funs == 1:
            # normalize each eigenfunction by its max value
            Wplot = WT[0,:]/max(abs(WT[0,:]))
            Tplot = WT[1,:]/max(abs(WT[1,:]))
            
            # plotting
            xmx = 1.2
            ymn = rspan[0]
            ymx = rspan[1]
            
            if nmax == 0:
                ax.plot(Wplot,rvec/1000,'b')    # W(r), displacement (blue)
                ax.plot(Tplot,rvec/1000,'r')    # T(r), stress (red)
                ax.plot([-xmx, xmx],ymn/1000*np.ones(2),'k')
                ax.plot([-xmx, xmx],ymx/1000*np.ones(2),'k')
                ax.plot(np.zeros(2),np.array([ymn, ymx])/1000,'k')
                
                ax.axis([-xmx, xmx, ymn/1000-300, ymx/1000+300])
                ax.set(title='f = %.2f mHz, T = %.2f min (l = %i)'% (froots[n]*1000,1/f0/60,l))
                ax.text(-1,ymx/1000+100,f'n = {n}')
            
                if n % 3 == 0:
                    ax.set(ylabel='radius (km)')
                    
            else:
                if n == 0 or n == 3 or n == 6: 
                    col = 0
                elif n == 1 or n == 4 or n == 7:
                    col = 1
                elif n == 2 or n == 5 or n == 8:
                    col = 2
                            
                if n == 0 or n == 1 or n == 2: 
                    row = 0
                elif n == 3 or n == 4 or n == 5:
                    row = 1
                elif n == 6 or n == 7 or n == 8:
                    row = 2
                    
                ax[row,col].plot(Wplot,rvec/1000,'b')    # W(r), displacement (blue)
                ax[row,col].plot(Tplot,rvec/1000,'r')    # T(r), stress (red)
                ax[row,col].plot([-xmx, xmx],ymn/1000*np.ones(2),'k')
                ax[row,col].plot([-xmx, xmx],ymx/1000*np.ones(2),'k')
                ax[row,col].plot(np.zeros(2),np.array([ymn, ymx])/1000,'k')
                
                ax[row,col].axis([-xmx, xmx, ymn/1000-300, ymx/1000+300])
                ax[row,col].set(title='f = %.2f mHz, T = %.2f min (l = %i)'% (froots[n][0]*1000,1/f0/60,l))
                ax[row,col].text(-1,ymx/1000+100,f'n = {n}')
            
                if n % 3 == 0:
                    ax[row,col].set(ylabel='radius (km)')
                    
        if n == nmax:   # leave the loop when you reach n=nmax
            break
        
        # advance to the next root (i.e., overtone)
        n += 1
        
plt.show()
print('/// l = %i, nroots = %i (nmax = %i, fmax = %.3f mHz)\n'% (l,sum(~np.isnan(froots))[0],nmax,fmax*1e3))

In [ ]:
# Toroidal mode observations in PREM
# Must have prem_Tmodes.txt in working directory

nobs, lobs, Tobs, Tobs_std = np.loadtxt('./data/prem_Tmodes_obs.txt',usecols=(0,2,3,4),unpack=True)
nobs = nobs.astype(int)
lobs = lobs.astype(int)
print('normal mode observations (measured from seismograms):')

for ii in range(len(nobs)):
    print('n = %i, l = %2i, T = %8.2f +/- %.2f s'% (nobs[ii],lobs[ii],Tobs[ii],Tobs_std[ii]))
    
# It may be useful to store them in a matrix
maxl = max(lobs)
maxn = max(nobs)
Tmat = np.full((maxn+1,maxl),np.nan)

for ii in range(len(nobs)):
    Tmat[nobs[ii],lobs[ii]-1] = Tobs[ii]

In [ ]:
# example code for calculating misfit in arrays with NaN
# say that A is observations, B is predictions

A = np.random.randn(4,10)
A[[2, 3, 0, 2],[0, 9, 3, 1]] = np.nan

B = np.random.randn(4,10)
B[[3, 1, 1],[7, 8, 2]] = np.nan

a = A.flatten(order='F')
b = B.flatten(order='F')
r = a - b

# You can't sum a vector that has NaN in it
print(sum(r**2))

# So just sum the values that are NOT NaN
dmis = sum(r[~np.isnan(r)])
print(dmis)